In [2]:
# https://freeprog.tistory.com/329

import sys
import IPython
import imutils
import cv2
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse


QApplication.setAttribute(Qt.AA_EnableHighDpiScaling, True)



class Left(QWidget): 
    w= 0
    h= 0
    pixelsPerMetric=0
    fname= './images/example_04.bmp'
    finame = ''
    sflist = []
    dir1= ''
    num = 0
    i= 0
    dB0=0
    def __init__(self):
    
        super().__init__()
        
        
        
        
        
        
        
        # 전체 폼 박스
        self.formbox = QHBoxLayout()
        self.setLayout(self.formbox)

        # 좌, 우 레이아웃박스
        self.left = QVBoxLayout()
        self.middle = QVBoxLayout()

        # 그룹박스1 생성 및 좌 레이아웃 배치
        gb = QGroupBox('그리기 종류')        
        self.left.addWidget(gb)

        # 그룹박스1 에서 사용할 레이아웃
        box = QVBoxLayout()
        gb.setLayout(box)        

        # 그룹박스 1 의 라디오 버튼 배치
        text = ['horizon']
        self.radiobtns = []

        for i in range(len(text)):
            self.radiobtns.append(QRadioButton(text[i], self))
            self.radiobtns[i].clicked.connect(self.radioClicked)
            box.addWidget(self.radiobtns[i])

        self.radiobtns[0].setChecked(True)
        self.drawType = 0
         
        # 그룹박스2
        gb = QGroupBox('펜 설정')        
        self.left.addWidget(gb)        

        grid = QGridLayout()      
        gb.setLayout(grid)        

        label = QLabel('선굵기')
        grid.addWidget(label, 0, 0)

        self.combo = QComboBox()
        grid.addWidget(self.combo, 0, 1)       

        for i in range(1, 21):
            self.combo.addItem(str(i))

        label = QLabel('선색상')

        grid.addWidget(label, 1,0)        
         
        self.pencolor = QColor(0,0,0)
        self.penbtn = QPushButton()        
        self.penbtn.setStyleSheet('background-color: rgb(0,0,0)')
        self.penbtn.clicked.connect(self.showColorDlg)
        grid.addWidget(self.penbtn,1, 1)
         

        # 그룹박스3
        gb = QGroupBox('붓 설정')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)

        label = QLabel('붓색상')
        hbox.addWidget(label)                

        self.brushcolor = QColor(255,255,255)
        self.brushbtn = QPushButton()        
        self.brushbtn.setStyleSheet('background-color: rgb(255,255,255)')
        self.brushbtn.clicked.connect(self.showColorDlg)
        hbox.addWidget(self.brushbtn)
        
        # 그룹박스4
        gb = QGroupBox('스케일')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.scalebtn = QPushButton()  
        self.scalebtn.clicked.connect(self.scalepnt)
        
        hbox.addWidget(self.scalebtn)
        self.scalebtn.setText('print')
        
        self.lbl_1 = QLabel("  ")
        hbox.addWidget(self.lbl_1)
        
        
        # 그룹박스4
        gb = QGroupBox('크기 측정')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.pixelpntbtn = QPushButton()  
        self.pixelpntbtn.clicked.connect(self.pixelpnt)
        
        hbox.addWidget(self.pixelpntbtn)
        self.pixelpntbtn.setText('print')
        
        self.lbl_2 = QLabel("  ")
        hbox.addWidget(self.lbl_2)
            
        # 그룹박스5
        gb = QGroupBox('불러오기')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)    
            
        self.openbtn = QPushButton("File Open")
        self.openbtn.clicked.connect(self.openbtnClicked)
        self.label = QLabel()

        hbox.addWidget(self.openbtn)
        hbox.addWidget(self.label)
        
         # 그룹박스6 
        gb = QGroupBox('불러오기 방법')        
        self.left.addWidget(gb)

        box2 = QHBoxLayout()
        gb.setLayout(box2)        

        text = ['*.확장자(unstack)', '개별(unstack)']
        self.radiobtns2 = []

        for i in range(len(text)):
            self.radiobtns2.append(QRadioButton(text[i], self))
            self.radiobtns2[i].clicked.connect(self.radio2Clicked)
            box2.addWidget(self.radiobtns2[i])

        self.radiobtns2[0].setChecked(True)
        self.loadType = 0
         
    
        #그룹박스 7
        gb=QGroupBox('처리할 목록')
        self.left.addWidget(gb)
        
        self.tb = QTextBrowser()
        self.tb.setAcceptRichText(True)
        self.tb.setOpenExternalLinks(True)

        self.next_btn = QPushButton('Nextr')
        self.next_btn.pressed.connect(self.next_view)
        self.clear_btn = QPushButton('Clear')
        self.clear_btn.pressed.connect(self.clear_text)

        hbox = QHBoxLayout()
        hbox.addWidget(self.tb, 1)
        hbox.addWidget(self.next_btn, 2)
        hbox.addWidget(self.clear_btn, 3)
        
        gb.setLayout(hbox)

        
        self.left.addStretch(1)
        
        
        
        # 전체 폼박스에 왼쪽 박스 배치
        self.formbox.addLayout(self.left)
        self.formbox.setStretchFactor(self.left, 0)

        self.openbtnClicked()#############파일 선택
#         self.fname= './images/example_04.bmp'
#         self.w= 300
#         self.h = 300
        
        
        
        
        
        
        
#         self.Graview()
        
#         (self.w,self.h)=self.Graview()
        print("Graview 나온 후의 w,h는 ", self.w, self.h)
#         self.middle.addWidget(self.view)
#          # 전체 폼박스에 오른쪽 박스 배치
#         self.formbox.addLayout(self.middle)

#         self.formbox.setStretchFactor(self.middle, 0)
        
        
        
        
        
#     def Graview(self):
#         print("Graview안에 있는 w,h는 ", self.w, self.h)
#         self.view =CView(self)
        
#         self.view.setFixedHeight(self.h)
#         self.view.setFixedWidth(self.w)
#         Left.h = self.h
#         Left.w = self.w
        
        #중간 레이아웃 박스에 그래픽 뷰 추가
        


                
        

        
        
        
        
        
        
         
    def radioClicked(self):
        for i in range(len(self.radiobtns)):
            if self.radiobtns[i].isChecked():
                self.drawType = i                
                break

    def checkClicked(self):
        pass
             
    def showColorDlg(self):       
         
        # 색상 대화상자 생성      
        color = QColorDialog.getColor()

        sender = self.sender()

        # 색상이 유효한 값이면 참, QFrame에 색 적용
        if sender == self.penbtn and color.isValid():           
            self.pencolor = color
            self.penbtn.setStyleSheet('background-color: {}'.format( color.name()))
        else:
            self.brushcolor = color
            self.brushbtn.setStyleSheet('background-color: {}'.format( color.name()))

    def scalepnt(self):
        global dy
#         global pixelsPerMetric
        
        
        self.txt_1=repr("{} ".format(Left.width)+chr(956)+"m")+"길이당 {:}".format(Left.dB0)+" pixels"    
        
        
        self.lbl_1.setText(self.txt_1)
        self.lbl_1.adjustSize()   
            
    def pixelpnt(self):
        global dy
#         global pixelsPerMetric
        print(Left.pixelsPerMetric)
        
        self.size= dy/Left.pixelsPerMetric
        
        self.txt_2="{}픽셀, ".format(dy)+repr("{:.5f} ".format(self.size)+chr(956)+"m")     
        
        
        self.lbl_2.setText(self.txt_2)
        self.lbl_2.adjustSize()    
        
#     def append_text(self):

#         textlist= self.tb.toPlainText()
#         print(type(textlist))
#         print(textlist)

    def clear_text(self) :

        self.tb.clear()

        
    def radio2Clicked(self):
        for i in range(len(self.radiobtns2)):
            if self.radiobtns2[i].isChecked():
                self.loadType = i                
                break
        
    
    def openbtnClicked(self):
        import os
        
        self.tuplefname = QFileDialog.getOpenFileName(self)
        self.fname = self.tuplefname[0]
        print(self.fname)
        dir,file = os.path.split(self.fname)
        print("1 ",dir, "에 있는 ", file)
#         상대경로로 쓰려고 할때 작업공간 계속 바꿔줬는데, 절대경로로 불러오면 안씀.
#   같은 위치 디렉토리 설정 에러 피하기 위해 동일한 디렉토리로는 변경하지 않음.
        self.dir1=dir
        if self.dir1==Left.dir1:
            None
        else:
            os.chdir(dir)
        Left.dir1=self.dir1
        
        print("현재 작업공간은 ",os.getcwd())
        self.label.setText(file)
        self.finame = file
        self.foname =os.path.basename(dir)
        self.flist = os.listdir(dir)
        print("2 ", self.flist)
        print("3 ", os.getcwd())
        
        #선택: 클릭한 확장자의 파일명을 클릭한 파일이 있는 폴더 내에서 모두 찾아 리스트 저장하기.(초기화 후)
        #선택: 클릭한 파일만 리스트에 저장하기(초기화 후)
        #텍스트브라우저에 나타내기
        search = os.path.splitext(file)[1]
        print("4 확장자는 ",search)
        
        self.sflist =[]
        self.tb.clear()
        if self.loadType == 0:
            
            
            for name in self.flist:
                if search in name:
                    self.sflist.append(name)
                    print("5 처리할 데이터 목록은 ", self.sflist)
                    self.tb.append(name)
                else:
                    None
            Left.sflist= self.sflist
                    
        else:
            self.sflist.append(file)
            print("5 처리할 데이터는 ", self.sflist)
            self.tb.append(file)
            
        Left.sflist = self.sflist
        Left.fname = self.fname
        Left.finame = self.finame
        #원래의 Graview내용
        print("Graview안에 있는 w,h는 ", self.w, self.h)
        Left.i = 0
        Left.changeView(self)
        
    def changeView(self):
        if Left.num ==0:
            None
        else:
            Left.deleteItems(self.middle)
#             self.view.deleteLater()
#             self.view = None
        
        self.view =CView(self)
        
        (Left.w,Left.h,Left.pixelsPerMetric,Left.width,Left.dB0)=ppp.ppp(Left.sflist[Left.i])
        
        Left.h = self.h
        Left.w = self.w
        
        
        self.view.setFixedHeight(self.h)
        self.view.setFixedWidth(self.w)
        self.middle.addWidget(self.view)
         # 전체 폼박스에 오른쪽 박스 배치
        self.formbox.addLayout(self.middle)

        self.formbox.setStretchFactor(self.middle, 0)
        print("이번에 띄울 이미지는 ",Left.i,"번째 이미지")
        self.setWindowTitle(self.sflist[Left.i])
        self.setGeometry(100,30,self.w+200,self.h)

        Left.scalepnt(self)
        Left.num +=1
        
        
    
    def deleteItems(layout): 
        if layout is not None: 
            while layout.count(): 
                item = layout.takeAt(0)
                print(item)
                widget = item.widget() 

                if widget is not None: 
                    widget.deleteLater() 
                else: 
                    deleteItems(item.layout()) 
    def next_view(self):
        if Left.i ==len(self.sflist)-1:
            Left.i=0
        else: 
            Left.i+=1
        Left.changeView(self)
    
        
# QGraphicsView display QGraphicsScene
class CView(QGraphicsView):
    pics =[]
    num = 0
    def __init__(self, parent):
        
        super().__init__(parent)   
        import os 
        
        self.dat= [Left.w,Left.h]
        print("CView로 받은 w,h 는 ",self.dat[:])
        
        print("CView에서의 작업공간은 ",os.getcwd())
        print("파일명은",Left.sflist[Left.i])
        
        
        self.scene = QGraphicsScene()
        

        pic = QPixmap('./'+Left.sflist[Left.i])
#         pic = QPixmap(Left.fname) # 위의 것보다 더 간단함.
#         CView.pics.append(pic)
#         CView.num=len(CView.pics)
#         print(CView.num)
#         print("이번에 불러올 이미지는 ", CView.num-1 ,"번째") 
        self.scene.addPixmap(pic)
        self.setScene(self.scene)
        
        
#         self.scene = QGraphicsScene()
#         #pic = QPixmap('./'+self.finame)
#         pic = QPixmap(Left.fname) # 위의 것보다 더 간단함.
#         CView.pics.append(pic)
#         CView.num=len(CView.pics)
#         print(CView.num)
#         print("이번에 불러올 이미지는 ", CView.num-1 ,"번째") 
#         self.scene.addPixmap(CView.pics[CView.num-1])
#         self.setScene(self.scene)
        
        
        """이 방법으로는 이미지 업데이트가 해결되지 않았다.
        img = QPixmap(Left.fname)
        self.pixmap = QGraphicsPixmapItem()
        self.pixmap.setPixmap(img)
        graphicsView = QGraphicsView(self)
        self.scene = QGraphicsScene()
        
        self.scene.addItem(self.pixmap)
        graphicsView.setScene(self.scene)
        """

        
        self.items = []
        
        self.start = QPointF()
        self.end = QPointF()
        self.setRenderHint(QPainter.HighQualityAntialiasing)  
        
#     @staticmethod    
#     def getwhfname(w,h,fname):
        
#         return w, h, fname
        
    def moveEvent(self, e): # 없애면 스크롤바 생김
        rect = QRectF(self.rect())
        rect.adjust(0,0,-2,-2)

        self.scene.setSceneRect(rect)
    
    def mousePressEvent(self, e):
        
        if e.buttons() & Qt.RightButton:           
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                del(self.items[-1])

        if e.button() == Qt.LeftButton:
            # 시작점 저장
            self.start = e.pos()
            self.end = e.pos() 
            
    
    def mouseMoveEvent(self, e):  
        
        if e.buttons() & Qt.RightButton:           
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                del(self.items[-1])
                
        # e.buttons()는 정수형 값을 리턴, e.button()은 move시 Qt.Nobutton 리턴 
        if e.buttons() & Qt.LeftButton:           

            self.end = e.pos()

            pen = QPen(self.parent().pencolor, self.parent().combo.currentIndex())

        
            # 수평선 그리기
            if self.parent().drawType == 0:
                 
                # 장면에 그려진 이전 선을 제거            
                if len(self.items) > 0:
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                
                hori1 = QLineF(0, self.start.y(), Left.w, self.start.y())
                
                self.items.append(self.scene.addLine(hori1, pen))
                
                hori2 = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                
                self.items.append(self.scene.addLine(hori2, pen))
                print(self.start.y(),self.end.y())
                
                
                
                """
                # 현재 선 추가
                hori = QLineF(self.start.x(), self.start.y(), self.end.x(), self.start.y())                
                self.items.append(self.scene.addLine(hori, pen))
                """

                
                
                
                
            
    
    def mouseReleaseEvent(self, e):  
        
        if e.button() == Qt.RightButton:
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                #del(self.items[-1])    

        if e.button() == Qt.LeftButton:

            pen = QPen(self.parent().pencolor, self.parent().combo.currentIndex())
            global dy 
            dy = abs(self.start.y()-self.end.y())
            self.dx = abs(self.start.x()-self.end.x())
            print(dy, self.dx)
            

    """
            if self.parent().drawType == 0:
                
                self.items.clear()
                hori = QLineF(0, self.start.y(), w, self.start.y())
                self.scene.addLine(hori, pen)
                hori = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                self.scene.addLine(hori, pen)
                print(self.start,self.end)
    """  

class ppp():
    
    @staticmethod
    def midpoint(ptA, ptB):
        return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)  
    @staticmethod
    def ppp(fname):
    
       
        # global fname
    #     fname = './images/example_04.bmp'
        width = 10.0





        # load the image, convert it to grayscale, and blur it slightly
        image =cv2.imread(fname)
        cv2.imshow('image',image)
        h, w = image.shape[:2]
        cv2.waitKey(0)
        h1=round(0.935*h)
        h2=round(0.97*h)
        w2=round(0.691*w)
        print(h1, h2, w2)
        image = image[h1:h2,0:w2]
        cv2.imshow('croped',image)
        cv2.waitKey(0)
        cv2.destroyAllWindows() 

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        blur = cv2.GaussianBlur(gray, (7, 7), 0)

        cv2.imshow('blur', blur)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # perform edge detection, then perform a dilation + erosion to
        # close gaps in between object edges
        canny = cv2.Canny(blur, 50, 100)
        edged = cv2.dilate(canny, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)

        # find contours in the edge map
        cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        cnts = imutils.grab_contours(cnts)

        # sort the contours from left-to-right and initialize the
        # 'pixels per metric' calibration variable
        (cnts, _) = contours.sort_contours(cnts)
    #     global pixelsPerMetric
        pixelsPerMetric = None

        # loop over the contours individually
        for c in cnts:
            # if the contour is not sufficiently large, ignore it
            if cv2.contourArea(c) < 100:
                continue

            # compute the rotated bounding box of the contour
            orig = image.copy()
            box = cv2.minAreaRect(c)
            box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            #print(box)

            # order the points in the contour such that they appear
            # in top-left, top-right, bottom-right, and bottom-left
            # order, then draw the outline of the rotated bounding
            # box
            box = perspective.order_points(box)
            cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

            # loop over the original points and draw them
            for (x, y) in box:
                cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)

            # unpack the ordered bounding box, then compute the midpoint
            # between the top-left and top-right coordinates, followed by
            # the midpoint between bottom-left and bottom-right coordinates
            (tl, tr, br, bl) = box
            (tltrX, tltrY) = ppp.midpoint(tl, tr)
            (blbrX, blbrY) = ppp.midpoint(bl, br)

            # compute the midpoint between the top-left and top-right points,
            # followed by the midpoint between the top-righ and bottom-right
            (tlblX, tlblY) = ppp.midpoint(tl, bl)
            (trbrX, trbrY) = ppp.midpoint(tr, br)

            # draw the midpoints on the image
            cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

            # draw lines between the midpoints
            cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
                (255, 0, 255), 2)
            cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
                (255, 0, 255), 2)

            # compute the Euclidean distance between the midpoints
            dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
            dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

            # if the pixels per metric has not been initialized, then
            # compute it as the ratio of pixels to supplied metric
            # (in this case, inches)
            if pixelsPerMetric is None:
                pixelsPerMetric = dB / width
                dB0=dB
            # compute the size of the object
            dimA = dA / pixelsPerMetric
            dimB = dB / pixelsPerMetric

            # draw the object sizes on the image
            cv2.putText(orig, "{:.5f}".format(dimB),
                (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)
            cv2.putText(orig, "{:.5f}".format(dimA),(int(trbrX + 10), int(trbrY)),
                cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)   


            # show the output image
            cv2.imshow("Image", orig)
            cv2.waitKey(0)
        cv2.destroyAllWindows()
        return w,h, pixelsPerMetric, width,dB0


# w=2000
# h=600


if __name__=="__main__":
    app = QApplication(sys.argv)
    ex = Left()
    ex.show()
    app.exec_()
    
app = IPython.Application.instance()
app.kernel.do_shutdown(True)
app.kernel.start()


C:/Users/Hojin/Anaconda3/envs/tensorflow/MYY/Measure/mouse_tracking/images/example_04.bmp
1  C:/Users/Hojin/Anaconda3/envs/tensorflow/MYY/Measure/mouse_tracking/images 에 있는  example_04.bmp
현재 작업공간은  C:\Users\Hojin\Anaconda3\envs\tensorflow\MYY\Measure\mouse_tracking\images
2  ['example_01.png', 'example_04.bmp']
3  C:\Users\Hojin\Anaconda3\envs\tensorflow\MYY\Measure\mouse_tracking\images
4 확장자는  .bmp
5 처리할 데이터 목록은  ['example_04.bmp']
Graview안에 있는 w,h는  0 0
CView로 받은 w,h 는  [0, 0]
CView에서의 작업공간은  C:\Users\Hojin\Anaconda3\envs\tensorflow\MYY\Measure\mouse_tracking\images
파일명은 example_04.bmp
957 993 884
이번에 띄울 이미지는  0 번째 이미지
Graview 나온 후의 w,h는  1280 1024
5.4


NameError: name 'dy' is not defined

264 265
264 267
264 269
264 273
264 280
264 292
264 303
264 318
264 329
264 339
264 349
264 356
264 363
264 368
264 372
264 376
264 378
114 34
5.4
C:/Users/Hojin/Desktop/계면활성제/실험군/190515 - 2000H Shot-peening with surfactant/190515/190515/8-25 down_15kV PC6 WD10_18.bmp
1  C:/Users/Hojin/Desktop/계면활성제/실험군/190515 - 2000H Shot-peening with surfactant/190515/190515 에 있는  8-25 down_15kV PC6 WD10_18.bmp
현재 작업공간은  C:\Users\Hojin\Desktop\계면활성제\실험군\190515 - 2000H Shot-peening with surfactant\190515\190515
2  ['8-25 down_15kV PC6 WD10_17.bmp', '8-25 down_15kV PC6 WD10_17.txt', '8-25 down_15kV PC6 WD10_18.bmp', '8-25 down_15kV PC6 WD10_18.txt', '8-25 down_15kV PC6 WD10_19.bmp', '8-25 down_15kV PC6 WD10_19.txt', '8-25 down_15kV PC6 WD10_20.bmp', '8-25 down_15kV PC6 WD10_20.txt', '8-25 down_15kV PC6 WD10_21.bmp', '8-25 down_15kV PC6 WD10_21.txt', '8-25 down_15kV PC6 WD10_22.bmp', '8-25 down_15kV PC6 WD10_22.txt', '8-25 down_15kV PC6 WD10_23.bmp', '8-25 down_15kV PC6 WD10_23.txt', '8-25 down_15kV PC6

957 993 884
이번에 띄울 이미지는  0 번째 이미지
295 297
295 299
295 300
295 303
295 306
295 308
295 311
295 314
295 316
295 318
295 319
295 321
295 323
295 324
295 325
295 326
295 327
295 328
295 330
295 331
295 332
295 333
295 334
295 336
41 10
5.4
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\계면활성제\실험군\190515 - 2000H Shot-peening with surfactant\190515\190515
파일명은 8-25 down_15kV PC6 WD10_18.bmp
957 993 884
이번에 띄울 이미지는  1 번째 이미지
C:/Users/Hojin/Desktop/계면활성제/실험군/190515 - 2000H Shot-peening with surfactant/190520/190520/8-28 down_20kV PC8 WD10_47.bmp
1  C:/Users/Hojin/Desktop/계면활성제/실험군/190515 - 2000H Shot-peening with surfactant/190520/190520 에 있는  8-28 down_20kV PC8 WD10_47.bmp
현재 작업공간은  C:\Users\Hojin\Desktop\계면활성제\실험군\190515 - 2000H Shot-peening with surfactant\190520\190520
2  ['8-28 down_20kV PC8 WD10_42.bmp', '8-28 down_20kV PC8 WD10_42.txt', '8-28 down_20kV PC8 WD10_43.bmp', '8-28 down_20kV PC8 WD10_43.txt', '8-28 down_20kV PC8 WD10_44.bmp', '8-28 down_20kV PC8 WD10_44.t